In [ ]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def classify_gesture(landmarks):
    finger_tips = [8, 12, 16, 20]   # Index to Pinky
    finger_pips = [6, 10, 14, 18]   # PIP joints
    finger_states = [landmarks[tip].y < landmarks[pip].y for tip, pip in zip(finger_tips, finger_pips)]

    thumb_extended = landmarks[4].x < landmarks[3].x
    index_extended = finger_states[0]
    middle_extended = finger_states[1]
    ring_extended = finger_states[2]
    pinky_extended = finger_states[3]

    if not any(finger_states) and not thumb_extended:
        return "Fist"
    elif index_extended and middle_extended and not ring_extended and not pinky_extended and not thumb_extended:
        return "Peace"
    elif all(finger_states):
        return "Hi"
    elif thumb_extended and pinky_extended and not any([index_extended, middle_extended, ring_extended]):
        return "YOLO"
    elif thumb_extended and index_extended and pinky_extended and not middle_extended and not ring_extended:
        return "Rockstar"
    else:
        return "Unknown"

cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = classify_gesture(hand_landmarks.landmark)
                text_size = cv2.getTextSize(gesture, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0]
                text_x = frame.shape[1] - text_size[0] - 10  # 10 pixels from right
                text_y = 40
                cv2.rectangle(frame,
                            (text_x - 5, text_y - text_size[1] - 5),
                            (text_x + text_size[0] + 5, text_y + 5),
                            (0, 0, 0), -1)
                cv2.putText(frame, gesture, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
                cv2.imshow("Gesture Recognition", frame)
                if cv2.waitKey(5) & 0xFF == 27:  
                    break

cap.release()
cv2.destroyAllWindows()